In [1]:
### インポート ###
import numpy as np
import pandas as pd
import glob
from selenium.webdriver.chrome.service import Service
from datetime import datetime
import time
import re
import csv
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, TypeVar, Generic
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [15]:
# =====
# 設定
# =====

### 設定ファイル ###
df_config_style = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="style", header=0)
df_config_min_records = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="min_records", header=0)
df_config_features = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="features", header=0)
df_config_target_courses = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="target_courses", header=0)
df_config_num_sumulation = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="num_sumulation", header=0)
# 脚質
STYLE_MAP = df_config_style.set_index('key')['value'].to_dict()
print(f'脚質: {STYLE_MAP}')
REVERSE_STYLE_MAP = {v: k for k, v in STYLE_MAP.items()}
# 最小許容有効レコード数
MIN_RECORDS_MAP = df_config_min_records.set_index('num_horses')['min_records'].to_dict()
print(f'最小許容有効レコード数: {MIN_RECORDS_MAP}')
# 最小騎手出走数
FEATURES_MAP = df_config_features.set_index('key')['value'].to_dict()
MIN_JOCKEY_RECORDS = FEATURES_MAP.get('min_jockey_records')
print(f'最小騎手出走数: {MIN_JOCKEY_RECORDS}')
# 対象コース
# 前準備：Excelのコース設定を、高速に検索できる「集合(set)」形式に変換
target_courses_set = set(
    df_config_target_courses[['racecourse', 'ground', 'distance', 'direction']]
    .itertuples(index=False, name=None)
)
print(f'対象コース: {target_courses_set}')
# シミュレーション回数
NUM_SIMULATIONS = df_config_num_sumulation.iloc[0, 0]
print(f'シミュレーション回数: {NUM_SIMULATIONS}')
### 共通オブジェクト ###
scaler = StandardScaler()

脚質: {1: '逃げ', 2: '先行', 3: '差し', 4: '追込'}
最小許容有効レコード数: {8: 8, 9: 9}
最小騎手出走数: 100
対象コース: {('園田', 'ダ', 1400, '右'), ('名古屋', 'ダ', 1500, '右'), ('高知', 'ダ', 1300, '右'), ('笠松', 'ダ', 1400, '右'), ('浦和', 'ダ', 1500, '左'), ('園田', 'ダ', 1230, '右'), ('船橋', 'ダ', 1500, '左'), ('笠松', 'ダ', 1600, '右'), ('川崎', 'ダ', 1400, '左'), ('船橋', 'ダ', 1200, '左'), ('水沢', 'ダ', 850, '右'), ('姫路', 'ダ', 1400, '右'), ('門別', 'ダ', 1000, '右'), ('佐賀', 'ダ', 1400, '右'), ('浦和', 'ダ', 1400, '左'), ('金沢', 'ダ', 1500, '右'), ('門別', 'ダ', 1200, '右'), ('川崎', 'ダ', 900, '左'), ('高知', 'ダ', 1400, '右'), ('水沢', 'ダ', 1400, '右'), ('佐賀', 'ダ', 1300, '右'), ('盛岡', 'ダ', 1200, '左')}
シミュレーション回数: 10000


In [3]:
# ========
# メソッド
# ========

def calculate_recent_time_index_avg(row, df):
    """
    特定の馬(horse_id)の直近2走の平均time_indexを計算する
    """
    target_id = row['horse_id']
    target_date = row['race_date']
    
    # 1. dfから同じhorse_id、かつ今回のrace_dateより前のデータを抽出
    # ※昇順にソート（直近が下に来るようにする）
    df_filtered = df[
        (df['horse_id'] == target_id) & 
        (df['race_date'] < target_date)
    ].sort_values('race_date')
    
    # 2. time_indexがNaNでないものを抽出
    df_valid = df_filtered.dropna(subset=['time_index'])
    
    # 3. 有効なデータが2つ以上あるか判定
    if len(df_valid) >= 2:
        # 下から2つ（直近2走）を選択して平均を出す
        avg_index = df_valid.tail(2)['time_index'].mean()
        return avg_index
    else:
        # 2回未満ならNaN
        return np.nan

def calculate_jockey_win_rate(row, df):
    """
    特定の騎手(jockey_id)の直近 MIN_JOCKEY_RECORDS 走（有効データ）の勝率を計算する
    """
    target_id = row['jockey_id']
    target_date = row['race_date']
    
    # 1. その騎手の、今回のレース日より前のデータを抽出
    # 2. かつ finish_rank が NaN でないものに絞る
    # 3. 日付順（昇順）にソート
    df_jockey_past = df[
        (df['jockey_id'].astype(str).str.zfill(5) == str(target_id).zfill(5)) & 
        (df['race_date'] < target_date) &
        (df['finish_rank'].notna())
    ].sort_values('race_date')
    
    # 4. 直近recent_num個のレコードを抽出（recent_num個なければ全件）
    df_recent = df_jockey_past.tail(MIN_JOCKEY_RECORDS)
    
    # レコード数をカウント（分母）
    total_count = len(df_recent)

    if total_count == 0:
        return np.nan
    
    # 5. finish_rank が 3 以下であるレコード数をカウント（分子）
    win_count = (df_recent['finish_rank'].astype(int) <= 3).sum()
    
    # 勝率を計算して返す
    return win_count / total_count

def predict_position_half_type(group):
    '''
    以下でソート
    1. 脚質順
    2. タイム指数順
    '''
    group = group.sort_values(["style_id", "time_index_avg_recent_2"], ascending=[True, False])
    n = len(group)
    mid = n // 2
    res = pd.Series(index=group.index, dtype=str)
    res.iloc[:mid] = "front"
    res.iloc[mid:] = "back"
    return res

def check_race_validity(group, target_courses):
    # --- 1. コース条件のチェック ---
    course_info = (
        group['racecourse'].iloc[0], 
        group['ground'].iloc[0], 
        group['distance'].iloc[0], 
        group['direction'].iloc[0]
    )
    if course_info not in target_courses:
        return False
    # --- 2. 頭数のチェック ---
    num_horses = group['num_horses'].iloc[0]
    if num_horses not in MIN_RECORDS_MAP:
        return False  
    # --- 3. 【今回の本題】特定の2つの変数による「レース丸ごと除外」チェック ---
    # 判定対象の列を絞り込む
    core_features = ['time_index_pred_from_race_avg', f'jockey_top3_rate_recent_{MIN_JOCKEY_RECORDS}']
    # 1つでも欠損（NaN）がある行が、そのグループ（レース）内に「存在する」か判定
    # any(axis=1)で「行に欠損があるか」、さらにany()で「グループ全体で1行でもあるか」を確認
    if group[core_features].isna().any(axis=1).any():
        return False
    # ここまで来れば、全頭の「タイム指数」と「騎手データ」が揃っていることが確定
    return True

def filter_valid_data(df, target_courses):
    """
    メインのフィルタリング実行関数
    """
    # 統合したチェック関数を適用
    # 引数を渡すために lambda を使用
    valid_mask = df.groupby('race_id').apply(
        lambda x: check_race_validity(x, target_courses)
    )
    # 有効なrace_idだけを抽出
    valid_race_ids = valid_mask[valid_mask].index
    df_filtered = df[df['race_id'].isin(valid_race_ids)].copy()
    
    return df_filtered

def create_dl_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid') 
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# =================
# 学習データ読み込み
# =================
print("trainデータ読み込み：開始")
df_train = pd.read_csv("C:\\keiba\\tool\\train\\train.csv", header=0, encoding='cp932')
print("trainデータ読み込み：終了")
### shutubaデータ読み込み ###
print("shutubaデータ読み込み：開始")
list_shutuba_files = glob.glob("C:\\keiba\\tool\\shutuba\\shutuba*.csv")
if list_shutuba_files:
    df_shutuba_raw = pd.read_csv(list_shutuba_files[0], header=0)
else:
    print("該当するファイルが見つかりませんでした。")
print("shutubaデータ読み込み：終了")

trainデータ読み込み：開始
trainデータ読み込み：終了
shutubaデータ読み込み：開始
shutubaデータ読み込み：終了


In [5]:
# =============
# 説明変数作成
# =============
print("説明変数作成：開始")
df_pred = df_shutuba_raw.copy()

### 予想タイム指数 ###
print("    予想タイム指数計算：開始")
# 直近過去2レースのタイム指数の平均値
df_pred['time_index_avg_recent_2'] = df_pred.apply(calculate_recent_time_index_avg, axis=1, args=(df_train,))
df_pred['time_index_avg_recent_2_race_avg'] = df_pred.groupby('race_id')['time_index_avg_recent_2'].transform('mean')
df_pred['time_index_pred_from_race_avg'] = df_pred['time_index_avg_recent_2'] - df_pred['time_index_avg_recent_2_race_avg']
# メモ：NaNをどう埋めるかは後で考える。front/back判定でも使う
print("    予想タイム指数計算：終了")

### 騎手複勝率 ###
print("    騎手複勝率計算：開始")
df_pred[f'jockey_top3_rate_recent_{MIN_JOCKEY_RECORDS}'] = df_pred.apply(calculate_jockey_win_rate, axis=1, args=(df_train,))
print("    騎手複勝率計算：終了")

### 対象レースに絞る
print("    対象レース絞り込み：開始")
df_train = filter_valid_data(df_train, target_courses_set)
df_train = df_train.dropna(subset=['finish_rank'])
df_pred = filter_valid_data(df_pred, target_courses_set)
print("    対象レース絞り込み：終了")

### コース×頭数×馬番別勝率
print("    コース×頭数×馬番別勝率計算：開始")
keys = ['racecourse', 'ground', 'distance', 'direction', 'num_horses', 'horse_number']
stats_df = df_train.groupby(keys)['finish_rank'].agg([
    ('win_count', lambda x: (pd.to_numeric(x, errors='coerce') == 1).sum()),
    ('total_count', 'count')
]).reset_index()
stats_df['condition_win_rate'] = stats_df['win_count'] / stats_df['total_count']
df_pred = pd.merge(df_pred, stats_df[keys + ['condition_win_rate']], on=keys, how='left')
print("    コース×頭数×馬番別勝率計算：終了")

### ポジションに紐づく勝率計算 ###
print("    ポジション（front / back）予想作成：開始")
df_pred["style_id"] = df_pred["style_pred"].map(REVERSE_STYLE_MAP)

df_pred = df_pred.sort_values(["race_id", "style_id", "time_index_avg_recent_2"], 
                             ascending=[True, True, False])
df_pred['temp_rank'] = df_pred.groupby("race_id").cumcount()
df_pred["position_half_type_pred"] = np.where(
    df_pred['temp_rank'] < (df_pred['num_horses'] // 2), 
    "front", 
    "back"
)
df_pred = df_pred.drop(columns=['temp_rank'])

print("    ポジション（front / back）予想作成：終了")

print("    予想ポジション別勝率計算：開始")

keys = ['racecourse', 'ground', 'distance', 'direction', 'num_horses', 'position_half_type_pred']
stats = df_train.groupby(keys)['finish_rank'].agg([
    ('win_count', lambda x: (pd.to_numeric(x, errors='coerce') == 1).sum()),
    ('total_count', 'count')
]).reset_index()
stats['position_half_type_win_rate'] = stats['win_count'] / stats['total_count']
# predデータのコース×頭数×(front/back)に紐づく勝率を割り当てる
df_pred = pd.merge(
    df_pred, 
    stats[keys + ['position_half_type_win_rate']], 
    on=keys, 
    how='left'
)
print("    予想ポジション別勝率計算：終了")

print("    予想ポジション比率別勝率計算：開始")

df_train['front_count'] = df_train.groupby('race_id')['position_half_type_pred'].transform(lambda x: (x == 'front').sum())
df_train['back_count'] = df_train.groupby('race_id')['position_half_type_pred'].transform(lambda x: (x == 'back').sum())
keys = ['num_horses', 'front_count', 'back_count', 'position_half_type_pred']
stats = df_train.groupby(keys)['finish_rank'].agg(
    composition_pos_win_rate = lambda x: (pd.to_numeric(x, errors='coerce') == 1).sum() / len(x)
).reset_index()
df_pred['front_count'] = df_pred.groupby('race_id')['position_half_type_pred'].transform(lambda x: (x == 'front').sum())
df_pred['back_count'] = df_pred.groupby('race_id')['position_half_type_pred'].transform(lambda x: (x == 'back').sum())
df_pred = pd.merge(df_pred, stats, on=keys, how='left')

print("    予想ポジション比率別勝率計算：終了")

print("説明変数作成：終了")

説明変数作成：開始
    予想タイム指数計算：開始
    予想タイム指数計算：終了
    騎手複勝率計算：開始
    騎手複勝率計算：終了
    対象レース絞り込み：開始


C:\Users\ryo\AppData\Local\Temp\ipykernel_13400\3093023001.py:106: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_mask = df.groupby('race_id').apply(
C:\Users\ryo\AppData\Local\Temp\ipykernel_13400\3093023001.py:106: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_mask = df.groupby('race_id').apply(


    対象レース絞り込み：終了
    コース×頭数×馬番別勝率計算：開始
    コース×頭数×馬番別勝率計算：終了
    ポジション（front / back）予想作成：開始
    ポジション（front / back）予想作成：終了
    予想ポジション別勝率計算：開始
    予想ポジション別勝率計算：終了
    予想ポジション比率別勝率計算：開始
    予想ポジション比率別勝率計算：終了
説明変数作成：終了


In [6]:
# ==========
# モデル作成
# ==========

print("モデル作成：開始")

### 説明変数 ###
features = [
    'time_index_pred_from_race_avg',
    f'jockey_top3_rate_recent_{MIN_JOCKEY_RECORDS}',
    'condition_win_rate',
    'position_half_type_win_rate',
    'composition_pos_win_rate'
]

### DeepLearningモデル作成 ###
dl_model = create_dl_model(len(features))

print("モデル作成：終了")

モデル作成：開始
モデル作成：終了


C:\Users\ryo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# =====
# 学習
# =====

print("学習：開始")

### 学習データ ###
X_train = df_train[features]
y_train = df_train['relative_rank_bin']
X_train_scaled = scaler.fit_transform(X_train)

### 学習 ###
dl_model.fit(X_train_scaled, y_train, epochs=20, batch_size=16, verbose=0)

print("学習：終了")

学習：開始
学習：終了


In [8]:
# =====
# 予想
# =====

print("予想：開始")

### 予想データ ###
X_pred = df_pred[features]
X_pred_scaled = scaler.transform(X_pred)

### 予想（上位半分に入る確率） ###
df_pred["pred_prob"] = dl_model.predict(X_pred_scaled, verbose=0).flatten()

print("予想：終了")

df_pred.to_csv("C:\\keiba\\tool\\pred\\pred.csv", index=False, encoding="cp932")

予想：開始
予想：終了


In [19]:
# ===============
# シミュレーション
# ===============

# 予想データのレースIDのリストを作成
list_race_id = df_pred['race_id'].unique().tolist()
# レースごとにループ処理
all_processed_races = []
for race_id in list_race_id:
    # 1. 抽出
    df_race = df_pred[df_pred['race_id'] == race_id].copy()
    df_pred_prob = df_race["pred_prob"].values
    h_nums = df_race["horse_number"].values
    # 2. シミュレーション
    dic_win_counts = {h: 0 for h in h_nums}
    for _ in range(NUM_SIMULATIONS):
        idx = np.argmax(df_pred_prob + np.random.normal(0, 0.09, len(df_pred_prob)))
        dic_win_counts[h_nums[idx]] += 1
    # 3. win_rateカラムを追加
    df_race['win_rate'] = df_race['horse_number'].map(lambda x: dic_win_counts[x] / NUM_SIMULATIONS)
    # 4. リストに追加
    all_processed_races.append(df_race)
# --- ループ終了後 ---
# 5. 全てのレースを一つのデータフレームに結合（これがマージに相当します）
df_pred = pd.concat(all_processed_races).reset_index(drop=True)

In [23]:
# ===============
# 出力
# ===============

df_pred.to_csv("C:\\keiba\\tool\\pred\\pred.csv", index=False, encoding="cp932")